## Test - EPA HUB

In [129]:
import re
import requests
import pandas as pd
from io import BytesIO
from bs4 import BeautifulSoup

### Parameters

In [4]:
URL = "https://www.epa.gov/climateleadership/ghg-emission-factors-hub"

### Functions

In [6]:
headers = {
    "Authority": "www.epa.gov",
    "Method": "GET",
    "Path": "/climateleadership/ghg-emission-factors-hub",
    "Scheme": "https",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Cache-Control": "max-age=0",
    "If-None-Match": "1689275341",
    "Sec-Ch-Ua": '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "cross-site",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}


### Tests

In [45]:
r = requests.get(URL, headers=headers)
soup = BeautifulSoup(r.content, "html.parser")
# Get all hrefs and their text
href_dicts = []
hrefs = soup.find_all("a")
for href in hrefs:
    href_dict = {}
    text=  href.text
    url = href.get("href")

    if "GHG" not in text:
        continue

    if "xls" in url:     
        href_dict["name"] = str(text).replace("\n", "")
        href_dict["url"] =  f"https://www.epa.gov{url}"
        href_dicts.append(href_dict)
 
    
    elif "pdf" in url:
        href_dict["name"] = str(text).replace("\n", "") 
        href_dict["url"] = f"https://www.epa.gov{url}"

        href_dicts.append(href_dict)

# Create dataframe
df = pd.DataFrame.from_dict(href_dicts)

In [46]:
df

,name,url
0,2023 GHG Emission Factors Hub (xlsx),https://www.epa.gov/system/files/documents/202...
1,2023 GHG Emission Factors Hub (pdf),https://www.epa.gov/system/files/documents/202...
2,ARCHIVED 2022 GHG Emission Factors Hub (xlsx),https://www.epa.gov/system/files/documents/202...
3,ARCHIVED 2022 GHG Emission Factors Hub (pdf),https://www.epa.gov/system/files/documents/202...
4,ARCHIVED 2021 GHG Emission Factors Hub (xlsx),https://www.epa.gov/system/files/documents/202...
5,ARCHIVED 2021 GHG Emission Factors Hub (pdf),https://www.epa.gov/system/files/documents/202...
6,ARCHIVED 2020 GHG Emission Factors Hub (xlsx),https://www.epa.gov/system/files/documents/202...
7,ARCHIVED 2020 GHG Emission Factors Hub (pdf),https://www.epa.gov/sites/default/files/2021-0...
8,ARCHIVED 2018 GHG Emission Factors Hub (xlsx),https://www.epa.gov/system/files/documents/202...
9,ARCHIVED 2018 GHG Emission Factors Hub (pdf),https://www.epa.gov/sites/default/files/2018-0...


### Getting xlsx files

In [137]:
xlsx_urls = df["url"].tolist()
undesired_words = ["pdf"]

xlsx_urls = [url for url in xlsx_urls if not any(word in url for word in undesired_words)]
xlsx_urls
first_url = xlsx_urls[2]
raw_year = re.findall(r"\-\d+\.", first_url)
raw_year = raw_year[0].replace("-", "").replace(".", "")
raw_year

first_url.split("/")[-1]

'ghg-emission-factors-hub-2021.xlsx'

In [125]:
mega_df = pd.DataFrame([])
for xlsx_url in xlsx_urls:
    response = requests.get(xlsx_url, headers=headers)
    xls_file = pd.ExcelFile(BytesIO(response.content))
    # Create a dictionary of DataFrames, with sheet name as key
    dataframes = {sheet_name: xls_file.parse(sheet_name) for sheet_name in xls_file.sheet_names}
    key_of_interest = None
    for key in dataframes.keys():
        if "Hub" in key:
            key_of_interest = key

    if key_of_interest is None:
        print(f"Skipping {xlsx_url}")
        continue

    ghg_data = dataframes[key_of_interest]
    ghg_data = ghg_data.dropna(axis=1, how="all")
    # Remove rows with several NaNs
    ghg_data = ghg_data.dropna(axis=0, thresh=2)
    ghg_data.reset_index(drop=True, inplace=True)

    mega_df = pd.concat([mega_df, ghg_data], ignore_index=True)

c:\Users\fdmol\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\fdmol\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\fdmol\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\fdmol\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\fdmol\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot par

In [128]:
mega_df.head(20)

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,NaN,Gas,100-Year GWP,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,CH4,25,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,N2O,298,NaN,NaN,NaN,NaN,NaN,NaN
3,Table 1,Stationary Combustion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Fuel Type,Heat Content (HHV),CO2 Factor,CH4 Factor,N2O Factor,CO2 Factor,CH4 Factor,N2O Factor,NaN
5,NaN,NaN,mmBtu per short ton,kg CO2 per mmBtu,g CH4 per mmBtu,g N2O per mmBtu,kg CO2 per short ton,g CH4 per short ton,g N2O per short ton,NaN
6,NaN,Anthracite Coal,25.09,103.69,11,1.6,2602,276,40,NaN
7,NaN,Bituminous Coal,24.93,93.28,11,1.6,2325,274,40,NaN
8,NaN,Sub-bituminous Coal,17.25,97.17,11,1.6,1676,190,28,NaN
9,NaN,Lignite Coal,14.21,97.72,11,1.6,1389,156,23,NaN


In [127]:
mega_df.to_csv("epa_hub_all.csv", index=False)